#### **Import des librairies.**

In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
""" Imports   """

from pyspark.sql import * 
from pyspark.sql.functions import * 
from pyspark.sql.types import *
import pandas as pd

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 3, Finished, Available, Finished)

##### **Chargement des données sources dans la couche Bronze**
##### 
##### - Collecte des fichiers dans le Lakehouse 
##### - Intégration et chargement dans les tables delta

In [2]:
"""" Loading the data into df """
salesSchema = StructType([
    StructField("Order_ID", StringType()),
    StructField("OrderDate", StringType()),
    StructField("ShippingDate", StringType()),
    StructField("Aging", IntegerType()),
    StructField("Ship_Mode", StringType()),
    StructField("Product_Category", StringType()),
    StructField("Product", StringType()),
    StructField("Sales", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("Discount", StringType()),
    StructField("Profit", StringType()),
    StructField("Shipping_Cost", StringType()),
    StructField("Order_Priority", StringType()),
    StructField("Customer_ID", StringType()),
    StructField("Customer_Name", StringType()),
    StructField("Segment", StringType()),
    StructField("City", StringType()),
    StructField("State", StringType()),
    StructField("Country", StringType()),
    StructField("Region", StringType()),
 ])


Sales = spark.read.format("csv").option("header","true").option("delimiter", ";").schema(salesSchema).load("Files/raw_data/Sales*.csv")
# df now is a Spark DataFrame containing CSV data from "Files/raw_data/Sales_01012023.csv".
display(Sales.limit(2))

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ce5d9c8f-ceb5-484b-afbb-b89b36f466fe)

In [3]:
Returns = spark.read.format("csv") \
                    .option("header","true") \
                    .option("delimiter", ";") \
                    .load("Files/raw_data/Return*.csv")
# df now is a Spark DataFrame containing CSV data from "Files/raw_data/Return_04012023.csv".
display(Returns)

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ad3e7a00-a465-400f-b4d7-da4860a12831)

In [4]:

final = Sales.join(Returns, Sales.Order_ID == Returns.Order_ID, "left").drop(Returns.Order_ID, Returns.Customer_Name, Returns.Sales_Amount)
display(final.limit(10))

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 144447e9-b536-4a04-bfde-d071b3a64c65)

In [5]:
final = final.withColumns(
            {
                "created_TS" : current_timestamp() , 
                "modified_TS" : current_timestamp()
            }
        )

display(final.limit(1))

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f4b1a6fb-f7be-417d-930c-3b27e0a6bd96)

In [7]:
%%sql

create table if not EXISTS bronze_layer (
    Order_ID string,
    OrderDate String,
    ShippingDate String,
    Aging int,
    Ship_Mode String, 
    Product_Category String,
    Product String,
    Sales String,
    Quantity int,
    Discount String,
    Profit String, 
    Shipping_Cost String,
    Order_Priority String,
    Customer_ID String,
    Customer_Name String,
    Segment String,
    City String,
    State String, 
    Country String,
    Region String,
    Return String,
    --Order_Year int,
    --Order_Month int,
    created_TS timestamp,
    modified_TS timestamp
)
USING DELTA

StatementMeta(, c90ddef3-f30d-40a4-9df1-3f22e8096344, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
%%sql
---drop table bronze_layer

StatementMeta(, b3eca0f6-fe2c-4f7f-9e77-eb70afeae526, 9, Finished, Available, Finished)

Error: 
[PARSE_SYNTAX_ERROR] Syntax error at or near end of input.(line 1, pos 26)

== SQL ==
---drop table bronze_layer
--------------------------^^^


In [8]:
final.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("bronze_layer") 

StatementMeta(, b3eca0f6-fe2c-4f7f-9e77-eb70afeae526, 10, Finished, Available, Finished)

In [8]:
%%sql
select * from bronze_layer 
where Return is not NULL
and Product is not NULL
limit 10; 

StatementMeta(, c90ddef3-f30d-40a4-9df1-3f22e8096344, 10, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 23 fields>

In [9]:
%%sql
SELECT * from bronze_layer where orderDate not regexp '^[0-9]{2}/[0-9]{2}/[0-9]{4}$' 

StatementMeta(, c90ddef3-f30d-40a4-9df1-3f22e8096344, 11, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 23 fields>

#### **Couche Silver** 
- Changement de la date :  du format jj-mois au format jj/mm/aaaa
- Suppression du charactère '$'
- Changement des types dedonnées : string -> Date, String -> Float

In [6]:
@udf
def formater_date(date):
    if '-' in date : 
        list_date = date.split("-")
        month= "00"
        year = "2022"
        if list_date[1] =='janv': month= "01" 
        elif list_date[1] =='févr': month= "02"
        elif list_date[1] =='mars': month= "03"
        elif list_date[1] =='avr': month= "04"
        elif list_date[1] =='mai': month= "05"
        elif list_date[1] =='juin': month= "06"
        elif list_date[1] =='juil': month= "07"
        elif list_date[1] =='aout': month= "08"
        elif list_date[1] =='sept': month= "09"
        elif list_date[1] =='oct': month= "10"
        elif list_date[1] =='nov': month= "11"
        elif list_date[1] =='nov': month= "12"

        return list_date[0]+ "/" + month + "/" + year
    else : 
        return date
    
@udf
def replace_character(val):
    return val.replace(',', '.')
   

@udf
def remove_character(val):
    replace = val.replace(',', '.')
    return replace.split('$')[1]



StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 8, Finished, Available, Finished)

In [11]:
test = to_date('02-02-2022')
display(test)

StatementMeta(, c90ddef3-f30d-40a4-9df1-3f22e8096344, 13, Finished, Available, Finished)

Column<'to_date(02-02-2022)'>

In [33]:
silver = final 
silver = silver.withColumns(
    {
        'orderDate': to_date(formater_date(silver['OrderDate']), "dd/mm/yyyy") ,
        'ShippingDate': to_date(formater_date(silver['ShippingDate']), "dd/mm/yyyy"),
        'Sales' :  remove_character(silver['Sales']).cast(FloatType()) , 
        'Profit' : remove_character(silver['Profit']).cast(IntegerType()) , 
        'Discount' : replace_character(silver['Discount']).cast(FloatType()),
        'Shipping_Cost' : remove_character(silver['Shipping_Cost']).cast(IntegerType()),
        'modified_TS' : current_timestamp()
    }
)

display(silver.limit(5))

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 39, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f66a29fc-455b-42b7-9dad-78c1f9590e46)

In [34]:
silver = silver.withColumns(
    {
        'OrderYear' : year(silver['orderDate']),
        'OrderMonth' : month(silver['orderDate']),
        'Return' : when(col('Return').isNull(), 0).otherwise(1) 
    }
)

display(silver.filter(col('Return') == 0).limit(2))

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 40, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5b8f472d-ae6c-49fd-80e8-84b93459dc74)

In [42]:
%%sql

create table if not EXISTS silver_layer (
    Order_ID string,
    OrderDate DATE,
    ShippingDate Date,
    Aging int,
    Ship_Mode String, 
    Product_Category String,
    Product String,
    Sales float,
    Quantity int,
    Discount float,
    Profit int, 
    Shipping_Cost int,
    Order_Priority String,
    Customer_ID String,
    Customer_Name String,
    Segment String,
    City String,
    State String, 
    Country String,
    Region String,
    Return int,
    OrderYear int,
    OrderMonth int,
    created_TS timestamp,
    modified_TS timestamp
)
USING DELTA

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 50, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [41]:
%%sql
drop table if exists silver_layer 

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 49, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [43]:
silver.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("silver_layer") 

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 51, Finished, Available, Finished)

In [44]:
%%sql
select * from silver_layer where Order_ID = 'AU-2015-30'

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 52, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 25 fields>

In [3]:
%%sql
select * from silver_layer 
where ShippingDate > OrderDate  

StatementMeta(, dff11dc1-9d70-452c-97a8-0fb38675c8b6, 4, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 23 fields>

In [16]:
%%sql
select count( distinct Customer_ID) from bronze_layer

StatementMeta(, c90ddef3-f30d-40a4-9df1-3f22e8096344, 18, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

#### **Couche Gold** 
- Création des tables de dimensions 
- Création de la table de fait 
- Calcul des KPIs 

In [24]:
""" Génération de la dimension Produit """
import uuid

@udf
def generate_id_prod():
    # Générer un UUID version 4 et en extraire les 4 derniers caractères
    random_part = str(uuid.uuid4())[-4:]
    return "PROD_" + random_part



@udf
def generate_id_ship():
    # Générer un UUID version 4 et en extraire les 4 derniers caractères
    random_part = str(uuid.uuid4())[-1:]
    return "SHIP_" + random_part

# Exemple d'utilisation
id_produit = generate_id_prod()
id_ship = generate_id_ship()
display(id_produit)  
display(id_ship) 

StatementMeta(, c90ddef3-f30d-40a4-9df1-3f22e8096344, 26, Finished, Available, Finished)

Column<'generate_id_prod()'>

Column<'generate_id_ship()'>

In [23]:
# Créer une table de dimension "produits"
df_produit = silver.select("Product", "Product_Category").distinct()

# Créer une table de dimension "clients"
df_client = silver.select("Customer_ID", "Customer_Name", "Segment", "City", "State", "Country", "Region").distinct()

# Créer la table de dimension "shipping"
df_shipping = silver.select("ShippingDate", "Ship_Mode", "Shipping_Cost").distinct()

StatementMeta(, c90ddef3-f30d-40a4-9df1-3f22e8096344, 25, Finished, Available, Finished)

In [34]:
""" Ajout des colonnes pour la dimension Produit """
df_produit = df_produit.withColumns(
    {
        'Product_ID' : generate_id_prod(),
        'created_TS' : current_timestamp(),
        'modified_TS' : current_timestamp()
    }
    
)
df_produit = df_produit.withColumnRenamed('Product', 'Product_Name')
display(df_produit)


df_client = df_client.withColumns(
    {
        'created_TS' : current_timestamp(),
        'modified_TS' : current_timestamp()
    }
    
)

#df_shipping = df_shipping.withColumn('Shipping_ID', generate_id_cust)

StatementMeta(, c90ddef3-f30d-40a4-9df1-3f22e8096344, 36, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c7d1fb32-3344-4d5d-b133-1732e2e17a14)

In [35]:
display(df_client)

StatementMeta(, c90ddef3-f30d-40a4-9df1-3f22e8096344, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bdd63b56-8c4d-4564-adfc-273cf416829b)

In [36]:
%%sql

create table if not EXISTS dim_produit (
    Product_ID String, 
    Product_Category String,
    Product String,
    created_TS timestamp,
    modified_TS timestamp
)
USING DELTA ; 




create table if not EXISTS dim_client (
    Customer_ID String, 
    Customer_Name String,
    Segment String,
    City String,
    State String,
    Country String,
    Region String,
    created_TS timestamp,
    modified_TS timestamp
)
USING DELTA ; 


create table if not EXISTS fact_order (
   Order_ID String
    , orderDate Date
    , OrderYear int
    , OrderMonth int
    , Order_Priority String
    , ShippingDate Date
    , Ship_Mode String
    , Shipping_Cost int 
    , UnitPrice Float
    , Quantity Int
    , Discount Float
    , Profit int
    , Return  int
    , Customer_ID String
    , Product_ID String
    , duree_livraison int 
    , CA float
    , Marge_Brute Float 
    ,cout_expedition float  
)
USING DELTA




StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 44, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [35]:
%%sql
drop table fact_order 

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 41, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [38]:
df_produit.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("dim_produit")
df_client.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("dim_client")  

StatementMeta(, c90ddef3-f30d-40a4-9df1-3f22e8096344, 41, Finished, Available, Finished)

In [45]:
%%sql
/* Creation de la table de fait  */
insert into fact_order 
select 
    /* Sélection des colonnes */
    Order_ID
    , orderDate
    , OrderYear
    , OrderMonth
    , Order_Priority
    , ShippingDate
    , Ship_Mode
    , Shipping_Cost 
    , Sales as UnitPrice
    , Quantity
    , Discount
    , Profit 
    , Return 
    , c.Customer_ID
    , p.Product_ID
   

    /* Calcul des KPIs  */
    , datediff(ShippingDate, orderDate) as duree_livraison
    , ( UnitPrice - (UnitPrice * Discount)) * Quantity as CA 
    , CA - Shipping_Cost as Marge_Brute 
    , Shipping_Cost / Quantity as Cout_expedition

from silver_layer s 
join dim_produit p on p.Product_Name = s.Product and p.Product_Category = s.Product_Category
join dim_client c on c.Customer_ID = s.Customer_ID



StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 53, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [46]:
%%sql
select * from fact_order where duree_livraison < 0  

StatementMeta(, 849b622a-31f6-4099-aa10-c2304f23ede0, 54, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 19 fields>

In [30]:
silver.withColumnRenamed('Sales','UnitPrice')

StatementMeta(, c90ddef3-f30d-40a4-9df1-3f22e8096344, 32, Finished, Available, Finished)

DataFrame[Order_ID: string, orderDate: date, ShippingDate: date, Aging: int, Ship_Mode: string, Product_Category: string, Product: string, UnitPrice: float, Quantity: int, Discount: float, Profit: int, Shipping_Cost: int, Order_Priority: string, Customer_ID: string, Customer_Name: string, Segment: string, City: string, State: string, Country: string, Region: string, Return: string, created_TS: timestamp, modified_TS: timestamp]